In [8]:
with open("input.txt", 'r') as f:
    input = f.read()

input1 = """029A
980A
179A
456A
379A"""

codes = input.split("\n")
codes

['083A', '935A', '964A', '149A', '789A']

In [9]:
def loc_dict(keypad: list):
    locs = {}
    for i, row in enumerate(keypad):
        for j, cell in enumerate(row):
            locs[cell] = (i,j)
    return locs

##############################
# +---+---+---+
# | 7 | 8 | 9 |
# +---+---+---+
# | 4 | 5 | 6 |
# +---+---+---+
# | 1 | 2 | 3 |
# +---+---+---+
#     | 0 | A |
#     +---+---+
# the magic value is 2 (intermediate value to avoid going to the empty key)
NUMERIC_KEYPAD = [["7", "8", "9"],
                  ["4", "5", "6"],
                  ["1", "2", "3"],
                  ["#", "0", "A"]]
NUMERIC_LOCS = loc_dict(NUMERIC_KEYPAD)

##############################
#     +---+---+
#     | ^ | A |
# +---+---+---+
# | < | v | > |
# +---+---+---+
# the magic value is v

DIRECTION_KEYPAD = [["#", "^", "A"],
                    ["<", "v", ">"]]
DIRECTION_LOCS = loc_dict(DIRECTION_KEYPAD)

#############################
KEYPADS = [NUMERIC_KEYPAD, DIRECTION_KEYPAD]
LOCATIONS = [NUMERIC_LOCS, DIRECTION_LOCS]

NUM_DIR_KEYPADS = 3
##############################


def get_complexity_sum(code_values: list, seq_lens: list):
    return sum([v*l for v, l in zip(code_values, seq_lens)])

    

### Part 1 & 2

In [10]:
CACHE = {}

DIRECTIONS = [(-1,0), # up
              (0,1),  # right
              (1,0),
              (0,-1)]

DIR_2_STR = {
    (-1,0): "^",
    (0,1): ">",
    (1,0): "v",
    (0,-1): "<",
}

def all_paths(prev, next, keypad_idx, max_steps=5):
    keypad = KEYPADS[keypad_idx]
    if prev == next: return [""]
    if max_steps == 0: return []
    output_paths = []
    for dir in DIRECTIONS:
        new_loc = prev[0]+dir[0], prev[1]+dir[1]
        if new_loc[0] not in range(len(keypad)) or new_loc[1] not in range(len(keypad[0])): continue
        if keypad[new_loc[0]][new_loc[1]] == "#": continue
        paths = all_paths(new_loc, next, keypad_idx, max_steps-1)
        output_paths.extend( [DIR_2_STR[dir] + path for path in paths] )
    return output_paths

def get_sequences(seq_in, keypad_idx, layer=3):
    keypad = KEYPADS[keypad_idx]
    locations = LOCATIONS[keypad_idx]
    if layer == 0:
        CACHE[(seq_in, layer)] = len(seq_in)
        return len(seq_in)
    if CACHE.get((seq_in, layer), -1) != -1:
        return CACHE[(seq_in, layer)]
    tot_len = 0
    for prev, next in zip("A"+seq_in, ("A"+seq_in)[1:]):
        prev_loc, next_loc = locations[prev], locations[next]
        ps = all_paths(prev_loc, next_loc, keypad_idx)
        outs, lens = [], []
        for p in ps:
            l = get_sequences(p+"A", keypad_idx=1, layer=layer-1)
            lens.append(l)
        min_len = min(lens)
        
        tot_len += min_len

    CACHE[(seq_in, layer)] = tot_len
    return tot_len

def all_codes_rec(codes: list[str], num_layers: int):
    global CACHE
    my_ls = []
    CACHE = {}
    for i, code in enumerate(codes):
        print(i)
        l = get_sequences(code, keypad_idx=0, layer=num_layers)
        my_ls.append(l)
    return my_ls


NUM_LAYERS = 26 # it was 3 in part 1, 26 in part 2
# get_sequences("<A",  keypad_idx=1, layer=1)
# get_sequences("029A", keypad_idx=0, layer=3)
lens = all_codes_rec(codes, num_layers=NUM_LAYERS)

code_values = [int(code[:-1]) for code in codes]
print(f"Sequence Lengths: {lens}")
print(f"Numeric Parts: {code_values}")
get_complexity_sum(code_values, lens)

0
1
2
3
4
Sequence Lengths: [80732180764, 86475783010, 85006969638, 91059074548, 80786362260]
Numeric Parts: [83, 935, 964, 149, 789]


246810588779586